# Identification de visages

**IMPORTANT :** Créez une ressource *Visage* ou *Face* et renseignez `SERVEUR` avec votre point de terminaison (endpoint). Récupérez également la clé et renseignez là lors de l'exécution de la première cellule.

In [ ]:
from getpass import getpass

CLE = getpass("Clé")
SERVEUR = 'https://face00.cognitiveservices.azure.com/'  # Votre point de terminaison

GROUPE = 'alice-bob-carol-dave'
PERSONNES = ['alice', 'bob', 'carol', 'dave']
ATTENTE = 26

## Préparation
### Création du groupe

In [ ]:
import requests

reponse = requests.put(
  url = f'{SERVEUR}/face/v1.0/persongroups/{GROUPE}',
  headers = {
    'Ocp-Apim-Subscription-Key': CLE,
    'Content-Type': 'application/json'
  },
  json = {
    'name': 'Alice, Bob, Carol et Dave',
    'recognitionModel': 'recognition_02'
  }
)
reponse

### Ajout de personnes au groupe

In [ ]:
import time

ids={}
for prenom in PERSONNES:
  essai = True
  while essai:
    reponse = requests.post(
      url = f'{SERVEUR}/face/v1.0/persongroups/{GROUPE}/persons',
      headers = {
        'Ocp-Apim-Subscription-Key': CLE,
        'Content-Type': 'application/json',
        'Accept': 'application/json'
      },
      json = { 'name': prenom }
    )
    if reponse.status_code==429:
      time.sleep(ATTENTE)
    else:
      essai = False
      identifiant = reponse.json()['personId']
      ids[identifiant] = prenom
      print(f'{prenom} ({identifiant}) :')
      for i in range(5):
        essai = True
        while essai:
          reponse = requests.post(
            url = f'{SERVEUR}/face/v1.0/persongroups/{GROUPE}/persons/{identifiant}/persistedFaces',
            headers = {
              'Ocp-Apim-Subscription-Key': CLE,
              'Content-Type': 'application/octet-stream'
            },
            data = open(f'data/{prenom}{i}.jpg', 'rb').read()
          )
          if reponse.status_code==429:
            time.sleep(ATTENTE)
          else:
            essai = False
            print("- %s%d.jpg : %s" % (prenom, i, 'ok' if reponse.status_code==200 else reponse.json()['error']['message']))

### Entraînement à l'identification des visages

In [ ]:
requests.post(
  url = f'{SERVEUR}/face/v1.0/persongroups/{GROUPE}/train',
  headers = {'Ocp-Apim-Subscription-Key': CLE}
)

### Destruction du groupe

In [ ]:
requests.delete(
  url = f'{SERVEUR}/face/v1.0/persongroups/{GROUPE}',
  headers = {'Ocp-Apim-Subscription-Key': CLE}
)

## Identification
### Appel API de détection

In [ ]:
reponse = requests.post(
  url = f'{SERVEUR}/face/v1.0/detect',
  headers = {
    'Ocp-Apim-Subscription-Key': CLE,
    'Content-Type': 'application/octet-stream',
    'Accept': 'application/json'
  },
  params = {'recognitionModel': 'recognition_02'},
  data=open('data/groupe.jpg', 'rb').read()
)
visages = reponse.json()
visages_id = [visage['faceId'] for visage in visages]

### Appel API d'identification

In [ ]:
reponse = requests.post(
  url = f'{SERVEUR}/face/v1.0/identify',
  headers = {'Ocp-Apim-Subscription-Key': CLE},
  json = {
    'personGroupId': GROUPE,
    'faceIds': visages_id,
    "maxNumOfCandidatesReturned": 1,
    "confidenceThreshold": 0.5
  }
)

### Traitement des résultats

In [ ]:
noms = dict.fromkeys(visages_id, '?')
for identite in reponse.json():
  if len(identite['candidates']) > 0:
    noms[identite['faceId']] = "%s %d%%" % (
      ids[identite['candidates'][0]['personId']], 
      identite['candidates'][0]['confidence']*100
    )

In [ ]:
from pprint import pprint

pprint(noms)

### Affichage sur la photo

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import patches

plt.figure(figsize=(20, 20))
img = plt.imshow(mpimg.imread('data/groupe.jpg'), alpha=0.6)
for v in visages:
  x, y, l, h = [v["faceRectangle"][k]
                for k in 'left top width height'.split(' ')]
  img.axes.add_patch(patches.Rectangle(
    (x, y), l, h, fill=False, linewidth=2, color='r'))
  plt.text(x, y, noms[v['faceId']], fontsize=15, weight="bold", va="bottom")
plt.axis("off")
